mine-tune an AI program to respond using the "brand voice".

In [5]:
%pip install dspy
%pip install ipywidgets
%pip install IPython
%pip install requests
%pip install firecrawl-py
%pip install beautifulsoup4
%pip install markdownify



You should consider upgrading via the '/Users/raymondweitekamp/.pyenv/versions/3.10.0/envs/my-env310/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/raymondweitekamp/.pyenv/versions/3.10.0/envs/my-env310/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/raymondweitekamp/.pyenv/versions/3.10.0/envs/my-env310/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/raymondweitekamp/.pyenv/versions/3.10.0/envs/my-env310/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/raymondweitekamp/.pyenv/versions/3.10.0/envs/my-env310/bin/python -m pip install --upg

In [7]:
import dspy
import os
import dotenv
import pydantic

dotenv.load_dotenv()
assert 'OPENAI_API_KEY' in os.environ
llm = dspy.OpenAI(model='gpt-4o', max_tokens=4096, temperature=0.1)
dspy.settings.configure(lm=llm)

set a URL directly or via input prompt

In [8]:
url = 'https://polyspectra.com'
#url = input("Enter a URL: ") 

preferred use firecrawl to get content more reliably. sign up at https://www.firecrawl.dev/ and put your API key in .env or directly below

In [10]:
from firecrawl import FirecrawlApp

# Initialize the FirecrawlApp with your API key
#app = FirecrawlApp(api_key='your_api_key')
FIRECRAWL_API_KEY = os.getenv("FIRECRAWL_API_KEY")
app = FirecrawlApp(api_key=FIRECRAWL_API_KEY)

# Scrape a single URL

scraped_data = app.scrape_url(url)
display(scraped_data)


{'content': '[polySpectra turns seven! Let’s celebrate.](https://polyspectra.com/blog/polyspectra-turns-seven)\n\n[![polySpectra | Make it real. | The strongest resin for 3D printing](https://polyspectra.com/images/logo_hu2f53e225e5f62e17c67df107ccbafcf3_3012_113x0_resize_box.png)](/)\n\n*   [COR Alpha](/alpha/)\n    \n*   [COR Black](/black/)\n    \n*   [COR Bio](/bio/)\n    \n*   [Docs](https://docs.polyspectra.com/)\n    \n*   [Shop](https://shop.polyspectra.com/)\n    \n\nSearch...\n\nCTRLK\n\n[Start Here](/start-here)\n\nMake It Real.\n\nRugged Resins For Industrial Additive Manufacturing\n===================================================\n\nWe help innovative engineers 3D print end-use components that they can trust, using the world’s most rugged photopolymer resins.\n\n[Start Here](/start-here)\n Feel the Strength\n\n![COR Alpha crushes the competition](https://polyspectra.com/images/COR-Alpha-Blender-450comp.webp)\n\n![Clients Image](https://polyspectra.com/images/trustedby/n

fallback to beautifulsoup if you don't want to sign up for firecrawl. many websites (like mine) block this type of scraping.

In [9]:
from bs4 import BeautifulSoup
import markdownify
import ipywidgets as widgets
from IPython.display import display
import requests

# Make a request to the URL
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Convert the HTML content to Markdown
markdown_content = markdownify.markdownify(str(soup), heading_style="ATX")

display(markdown_content)




'\nJust a moment...Enable JavaScript and cookies to continue'

choose what we are going to use as context

In [ ]:
from IPython.display import display, Markdown
context = scraped_data['markdown']
#context = markdown_content # or use beautifulsoup to convert to markdown
display(Markdown(context))



now we need a DSPy signature & module, and test it uncompiled (zero shot)

In [19]:
class BrandVoice(dspy.Signature):
    """Always respond to the user using the brand voice."""

    context = dspy.InputField(desc="This is a website of the brand that you are representing. Carefully analyze this to make sure your responses are branded and professional.")
    query = dspy.InputField()
    response = dspy.OutputField(desc="This is the response to the user's query. Always use the brand voice.")


class BrandAmbassador(dspy.Module):  # let's define a new module
    def __init__(self):
        super().__init__()
       
        self.signature = BrandVoice
        self.predictor = dspy.ChainOfThought(self.signature) #this can easily be subbed for something else
    
    def forward(self, query, context):
        result = self.predictor(query=query, context=context)
        return dspy.Prediction(response=str(result.response))


uncompiled = BrandAmbassador()
test = uncompiled("write an executive summary of polySpectra", context)
display(Markdown(test.response))


polySpectra Executive Summary

polySpectra is at the forefront of industrial additive manufacturing, empowering innovative engineers to 3D print end-use components with unparalleled confidence. Our mission is to "Make it Real" by providing the world's most rugged photopolymer resins, known for their exceptional strength, durability, and impact resistance. 

Our flagship products, including COR Alpha, COR Black, and COR Bio, are engineered to meet the most demanding industrial applications. These Cyclic Olefin Resins (COR) offer best-in-class thermomechanical performance, chemical resistance, and long-term durability, making them the go-to choice for engineers frustrated with brittle 3D printed prototypes.

polySpectra's commitment to excellence is reflected in our streamlined 3-step printing process: Print, Rinse, and Bake. This process ensures that our resins are not only easy to use but also deliver superior results. Our proprietary solutions, such as supaRinse cleaning fluid and waveCure heat transfer fluid, further enhance the user experience by simplifying post-processing.

Trusted by industry leaders like NASA, Dayco, and Asiga, polySpectra has established itself as a premier partner in the 3D printing landscape. Our resins have been featured in prominent publications and platforms, including Netflix, Forbes, and Engineering.com, underscoring our impact on the field of additive manufacturing.

At polySpectra, we don't just provide materials; we offer comprehensive services, including 3D printed parts and Design for Additive Manufacturing (DfAM) expertise. Our team of engineers is dedicated to helping clients overcome the common pitfalls of traditional manufacturing, ensuring that every component produced is robust and reliable.

Join us in revolutionizing 3D printing with the most rugged photopolymer resins on the market. Feel the strength of COR and make your innovations real with polySpectra.

Make it real. No more broken parts.